In [7]:
import trafilatura

In [53]:
# Political leaders to set out stall and start campaigning ahead of election PAIR
url_1 = "https://www.standard.co.uk/news/politics/keir-starmer-rishi-sunak-prime-minister-labour-jeremy-hunt-b1129944.html"

url_2 = "https://www.thetelegraphandargus.co.uk/news/national/24023150.political-leaders-set-stall-start-campaigning-ahead-election/"


# Late Queen ‘infuriated’ by Harry and Meghan’s claim over naming of daughter Lilibet PAIR
url_3 = "https://uk.news.yahoo.com/queen-infuriated-harry-meghan-claim-104006416.html?guccounter=1"

url_4 = "https://www.independent.co.uk/life-style/royal-family/harry-meghan-lilibet-robert-hardman-king-biography-b2478702.html"

In [54]:
urls = [url_1, url_2, url_3, url_4]

extracted_texts = {}

# Loop through each URL
for url in urls:
    downloaded = trafilatura.fetch_url(url)
    extracted_text = trafilatura.extract(
        downloaded,
        favour_recall=True,
        include_comments=False,
        include_images=False,
        include_tables=False
    )

    extracted_texts[url] = extracted_text

for url, text in extracted_texts.items():
    print(f"URL: {url}\nExtracted Text: {text}\n{'='*50}")

URL: https://www.standard.co.uk/news/politics/keir-starmer-rishi-sunak-prime-minister-labour-jeremy-hunt-b1129944.html
Extracted Text: Political party leaders are set to lay out their stall to voters this week as they mark the start of what is likely to be a general election year.
With 2024 expected to see voters go to the polls to elect a new Westminster government, the first week of January will see a flurry of activity.
Rishi Sunak is expected to tell voters he is on a tax reducing mission during a visit on Thursday as he marks the introduction of the national insurance cut.
The Prime Minister is set to meet people and take questions in the East Midlands during his first public appearance of the year, a No 10 source confirmed to the PA news agency.
The source said the Conservative Party leader would be reiterating his belief in “low taxes” and will point to how he has fired the starting gun on cutting taxes, starting with Saturday’s reduction in national insurance.
The Prime Ministe

In [10]:
#!pip install ppdeep

# Fuzzy hash for similarity > 80 means highly similar or near-duplicates 
# There is variance in trafilatura extraction due to dense web page content so exact match seems 
# unlikely.

import ppdeep

In [61]:
hash_values = {}

for url, text in extracted_texts.items():
    hash_value = ppdeep.hash(text)

    hash_values[url] = hash_value

In [70]:
comparison_result_1_2 = ppdeep.compare(hash_values[url_1], hash_values[url_2])
print(f"Comparison Result for Articles 1 and 2: {comparison_result_1_2}")

comparison_result_1_3 = ppdeep.compare(hash_values[url_1], hash_values[url_3])
print(f"Comparison Result for Articles 1 and 3: {comparison_result_1_3}")

comparison_result_2_3 = ppdeep.compare(hash_values[url_2], hash_values[url_3])
print(f"Comparison Result for Articles 2 and 3: {comparison_result_2_3}")

comparison_result_3_4 = ppdeep.compare(hash_values[url_3], hash_values[url_4])
print(f"Comparison Result for Articles 3 and 4: {comparison_result_3_4}")

comparison_result_1_4 = ppdeep.compare(hash_values[url_1], hash_values[url_4])
print(f"Comparison Result for Articles 1 and 4: {comparison_result_1_4}")

comparison_result_2_4 = ppdeep.compare(hash_values[url_2], hash_values[url_4])
print(f"Comparison Result for Articles 2 and 4: {comparison_result_2_4}")


Comparison Result for Articles 1 and 2: 88
Comparison Result for Articles 1 and 3: 0
Comparison Result for Articles 2 and 3: 0
Comparison Result for Articles 3 and 4: 97
Comparison Result for Articles 1 and 4: 0
Comparison Result for Articles 2 and 4: 0


This is providing exactly what I need really. Zero where we would expect and strong positives for 
duplicates!

In [16]:
!pip install mmh3
#  Non-cryptographic, speedy hash MurmurHash3
import mmh3

In [71]:
mmh3_hash_values = {}

for url, hash_value in hash_values.items():
    mmh3_hash_value = mmh3.hash(hash_value)
    mmh3_hash_values[url] = mmh3_hash_value


for url, mmh3_hash_value in mmh3_hash_values.items():
    print(f"URL: {url}\nmmh3 Hash Value: {mmh3_hash_value}\n{'='*50}")

URL: https://www.standard.co.uk/news/politics/keir-starmer-rishi-sunak-prime-minister-labour-jeremy-hunt-b1129944.html
mmh3 Hash Value: -1814420583
URL: https://www.thetelegraphandargus.co.uk/news/national/24023150.political-leaders-set-stall-start-campaigning-ahead-election/
mmh3 Hash Value: -2050602850
URL: https://uk.news.yahoo.com/queen-infuriated-harry-meghan-claim-104006416.html?guccounter=1
mmh3 Hash Value: 1032684053
URL: https://www.independent.co.uk/life-style/royal-family/harry-meghan-lilibet-robert-hardman-king-biography-b2478702.html
mmh3 Hash Value: -355670058


So storing a stronger hash is probably pointless as it is unlikely to ever be an identical match?

In [43]:
import nltk
# nltk.download('punkt')

def calculate_ttr(text):
    tokens = nltk.word_tokenize(text)
    types = set(tokens)
    ttr = len(types) / len(tokens) if len(tokens) > 0 else 0

    return ttr

In [72]:
ttr_values = {}

for url, text in extracted_texts.items():
    ttr_value = calculate_ttr(text)
    ttr_values[url] = ttr_value

for url, ttr_value in ttr_values.items():
    print(f"URL: {url}\nTTR Value: {ttr_value}\n{'='*50}")

URL: https://www.standard.co.uk/news/politics/keir-starmer-rishi-sunak-prime-minister-labour-jeremy-hunt-b1129944.html
TTR Value: 0.4238845144356955
URL: https://www.thetelegraphandargus.co.uk/news/national/24023150.political-leaders-set-stall-start-campaigning-ahead-election/
TTR Value: 0.44291091593475534
URL: https://uk.news.yahoo.com/queen-infuriated-harry-meghan-claim-104006416.html?guccounter=1
TTR Value: 0.4006677796327212
URL: https://www.independent.co.uk/life-style/royal-family/harry-meghan-lilibet-robert-hardman-king-biography-b2478702.html
TTR Value: 0.3962558502340094


TTR alone won't differentiate, but could it be a contributory factor in identifying suspicions?

In [80]:
from lexicalrichness import LexicalRichness
import pandas as pd

# Assuming you have the extracted_texts dictionary containing URLs as keys and article texts as values
# extracted_texts = {url_1: article_text_1, url_2: article_text_2, ...}

measures = [
    "words",
    "terms",
    "ttr",
    "rttr",
    "cttr",
    "msttr",
    "mattr",
    "mtld",
    "hdd",
    "vocd",
    "Herdan",
    "Summer",
    "Dugast",
    "Maas",
    "yulek",
    "yulei",
    "herdanvm",
    "simpsond",
]


lexical_richness_data = {"Measure": measures}

# Looping through each URL and calculate lexical richness measures
for url, text in extracted_texts.items():
    lex = LexicalRichness(text)
    lexical_richness_data[url] = [getattr(lex, measure)() if callable(getattr(lex, measure)) else getattr(lex, measure) for measure in measures]

df = pd.DataFrame(lexical_richness_data)

df.columns = ["Measure"] + [col.split("/")[-1] for col in df.columns[1:]]

df.rename(columns={
    "keir-starmer-rishi-sunak-prime-minister-labour-jeremy-hunt-b1129944.html": "Article 1",
    "queen-infuriated-harry-meghan-claim-104006416.html?guccounter=1": "Article 3",
    "": "Article 2",
    "harry-meghan-lilibet-robert-hardman-king-biography-b2478702.html": "Article 4"
}, inplace=True)

print(df.to_string(index=False))


 Measure  Article 1  Article 2  Article 3  Article 4
   words 675.000000 705.000000 497.000000 532.000000
   terms 305.000000 335.000000 235.000000 247.000000
     ttr   0.451852   0.475177   0.472837   0.464286
    rttr  11.739455  12.616830  10.541191  10.708808
    cttr   8.301049   8.921446   7.453747   7.572271
   msttr   0.713333   0.717143   0.720000   0.706000
   mattr   0.720660   0.724422   0.742714   0.737206
    mtld  89.574053  91.388545 103.709258 100.280922
     hdd   0.837117   0.841448   0.856829   0.859342
    vocd  90.744643  93.597669 107.772579 110.014515
  Herdan   0.878060   0.886544   0.879360   0.877760
  Summer   0.930611   0.935969   0.929592   0.929018
  Dugast  53.425771  57.803857  51.463762  51.347004
    Maas   0.018718   0.017300   0.019431   0.019475
   yulek 124.049383 119.833006 106.797728 104.231443
   yulei  15.447526  17.740278  19.043103  18.859042
herdanvm   0.102994   0.102062   0.091851   0.090853
simpsond   0.012423   0.012000   0.010701   0.

In [85]:
percentage_diff_1_2 = ((df["Article 1"] - df["Article 2"]) / df["Article 2"]) * 100
percentage_diff_1_3 = ((df["Article 1"] - df["Article 3"]) / df["Article 3"]) * 100
percentage_diff_2_4 = ((df["Article 2"] - df["Article 4"]) / df["Article 4"]) * 100
percentage_diff_3_4 = ((df["Article 3"] - df["Article 4"]) / df["Article 4"]) * 100

# Display the results
result_df = pd.DataFrame({
    "Measure": df["Measure"],
    "Percentage Difference (1-2)": percentage_diff_1_2,
    "Percentage Difference (1-3)": percentage_diff_1_3,
    "Percentage Difference (2-4)": percentage_diff_2_4,
    "Percentage Difference (3-4)": percentage_diff_3_4,
})

# Take absolute percentage change here
result_df[["Percentage Difference (1-2)", "Percentage Difference (1-3)", "Percentage Difference (2-4)", "Percentage Difference (3-4)"]] = result_df[["Percentage Difference (1-2)", "Percentage Difference (1-3)", "Percentage Difference (2-4)", "Percentage Difference (3-4)"]].abs()


print(result_df)

     Measure  Percentage Difference (1-2)  Percentage Difference (1-3)  \
0      words                     4.255319                    35.814889   
1      terms                     8.955224                    29.787234   
2        ttr                     4.908789                     4.438140   
3       rttr                     6.954001                    11.367452   
4       cttr                     6.954001                    11.367452   
5      msttr                     0.531208                     0.925926   
6      mattr                     0.519410                     2.969361   
7       mtld                     1.985470                    13.629646   
8        hdd                     0.514699                     2.300548   
9       vocd                     3.048181                    15.799878   
10    Herdan                     0.956913                     0.147776   
11    Summer                     0.572453                     0.109628   
12    Dugast                     7.574

In [86]:
df.to_csv('lexical_richness_comparison.csv', index=False)
result_df.to_csv('lexical_richness_comparison_%_difference.csv', index=False)